# Environment Setup

In [ ]:
#### download nltk packages
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
#### set up openai API key
%env OPENAI_API_KEY=PUT_YOUR_KEY_HERE

# Tutorial

### 1. Specify Models to Test

First, you can add models to be tested and also how you want to generate test data. Currently, we support OpenAI models only. 

If you want to extract Weaver's concepts without doing testing, you can skip this step.

In [ ]:
#### add model to be tested
from knowtest.specs.manager import SpecManager
model_spec = {
    "email_assistant": {
        "model": "gpt-3.5-turbo",
        "type": "generation",
        "prompt": [
            {
                "role": "user",
                "content": "You are an email assistant. Help reply to this email: {example}\n"
            }
        ]
    },
    "monologue_summarizer": {
        "model": "gpt-3.5-turbo",
        "type": "generation",
        "prompt": [
            {
                "role": "user",
                "content": "Summarize the monologue to key steps for future use: {example}\n"
            }
        ]
    }
}
SpecManager.add_model_specs(model_spec, overwrite=True)

In [ ]:
#### add test case generator
generator_specs = {
    "zero_shot_email": "You are a senior researcher. Write one email on '{topic}' to a student:\n",
    "few_shot_email": ''''You are a senior researcher. Write one email on '{topic}' to a student:

Examples:
{examples}

Your email:
''',
    
    "zero_shot_monologue": '''Write a think-aloud monologue of a person searching for an item in a library. The monologue should contain more than 10 sentences. The person should face some challenges when searching.
Scenario: {topic}
Format: [time] sentence
Example: [00:00] Where is the book?''',
    "few_shot_monologue": '''Write a think-aloud monologue of a person searching for an item in a library. The monologue should contain more than 10 sentences. The person should face some challenges when searching.
Scenario: {topic}

Examples:
{examples}

Your monologue:'''
}
    
SpecManager.add_generator_specs(generator_specs, overwrite=True)

### 2. Specify Weaver Relations

By default, we use ConceptNet relations, but you can optionally select a subset of them or use your own relations.

In [ ]:
#### [optional] use only custom relations
from knowtest.specs.manager import SpecManager
rel_specs = [
    "TYPEOF",
    'PARTOF',
    'OBSTRUCTEDBY',
    'LOCATEDNEAR',
    'CAUSEDBY',
    'RELATEDTO',
    {
        "id": "HASCHALLENGE",
        "tag": "has challenges",
        "description": {
            "text": "is a challenge of",
            "position": "second"
        },
        "prompt": "List {N} challenges of {topic}."
    },
]
SpecManager.specify_custom_relation_specs(rel_specs, overwrite=True)

### 3. Start Backend Server

In [ ]:
from knowtest.server.Capability import Capability

##### Option 1: start the server with models to test

In [ ]:
# start the server with models to test
c = Capability(topic="researcher emails", 
               model_spec="email_assistant", 
               uid="test", 
               generator_specs={
                   "zero_shot_data_gen": "zero_shot_email",
                   "few_shot_data_gen": "few_shot_email"
               }, 
               serverHost='localhost')

##### Option 2: start the server without models to test (Weaver concept only)

In [ ]:
# OR, start the server without models to test
c = Capability(topic="researcher emails", 
               model_spec="dummy", 
               uid="test", 
               serverHost='localhost')

### 4. Play with Frontend UI

In [ ]:
c.display()

### 5. Switch to different seed topics, models, or test_gen

In [ ]:
# change seed topic
# update models and generators optionally
c.change_topic(topic="book finding", 
               model_spec="monologue_summarizer",
               generator_specs={
                   "zero_shot_data_gen": "zero_shot_monologue",
                   "few_shot_data_gen": "few_shot_monologue"
               })

In [ ]:
c.display()